1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,Discount,0
63996,5,38.91,0,1,Urban,1,Phone,Discount,0
63997,6,29.99,1,0,Urban,1,Phone,Discount,0
63998,1,552.94,1,0,Surburban,1,Multichannel,Buy One Get One,0


In [4]:
df = df.rename(columns={'conversion': 'target', 'offer': 'treatment'})

In [7]:
df = df.replace({'Buy One Get One':1, 'Discount':1, 'No Offer':0})

In [8]:
df

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,1,0
63996,5,38.91,0,1,Urban,1,Phone,1,0
63997,6,29.99,1,0,Urban,1,Phone,1,0
63998,1,552.94,1,0,Surburban,1,Multichannel,1,0


3. сделать разбиение набора данных не тренировочную и тестовую выборки

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
df.columns.to_list()

['recency',
 'history',
 'used_discount',
 'used_bogo',
 'zip_code',
 'is_referral',
 'channel',
 'treatment',
 'target']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(df[['recency',
 'history',
 'used_discount',
 'used_bogo',
 'zip_code',
 'is_referral',
 'channel',
 'treatment']], df['target'], random_state=42)

In [61]:
train_treat = X_train['treatment']
test_treat = X_test['treatment']

In [62]:
X_train = X_train.drop('treatment', axis=1)
X_test = X_test.drop('treatment', axis=1)

In [63]:
X_train

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel
7517,6,92.45,0,1,Urban,0,Web
5343,1,796.97,0,1,Rural,1,Multichannel
35537,7,390.14,0,1,Urban,1,Phone
63983,2,83.03,0,1,Urban,0,Phone
45287,10,734.90,1,0,Rural,1,Multichannel
...,...,...,...,...,...,...,...
62570,3,276.21,0,1,Urban,0,Phone
38158,4,43.49,0,1,Urban,0,Phone
860,8,492.53,1,1,Rural,1,Phone
15795,5,72.28,1,0,Urban,1,Web


4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

In [64]:

X_train['hist/recency'] = X_train['history']/X_train['recency']
X_test['hist/recency'] = X_test['history']/X_test['recency']

In [65]:
X_train['hist*recency'] = X_train['history']*X_train['recency']
X_test['hist*recency'] = X_test['history']*X_test['recency']

In [66]:
X_train['hist/hist*recency'] = X_train['history']/X_train['hist*recency']
X_test['hist/hist*recency'] = X_test['history']/X_test['hist*recency']

In [67]:
scale_features = ['recency', 'history','hist/recency', 'hist*recency', 'hist/hist*recency']

In [68]:
from sklearn.preprocessing import StandardScaler

In [69]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train[scale_features]),columns = scale_features)

In [87]:
X_test_scaled = pd.DataFrame(scaler.transform(X_test[scale_features]),columns = scale_features)

In [88]:
X_train_scaled.index = X_train.index
X_test_scaled.index = X_test.index



In [90]:
X_train[scale_features]  = X_train_scaled
X_test[scale_features]  = X_test_scaled

In [93]:
cat_features = ['zip_code', 'channel']

5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

In [95]:
pip install scikit-uplift

Note: you may need to restart the kernel to use updated packages.


In [96]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel
from catboost import CatBoostClassifier

In [97]:
sm = SoloModel(CatBoostClassifier(iterations=100, thread_count=10, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, train_treat, estimator_fit_params={'cat_features':cat_features})


In [111]:
models_results = {
    'approach': [],
    'uplift@10%': [], 
    'uplift@20%': []
}

In [99]:
uplift_sm = sm.predict(X_test)

In [113]:
sm_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=test_treat, strategy='by_group', k=0.1)
models_results['approach'].append('SoloModel')
models_results['uplift@10%'].append(sm_score)
sm_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=test_treat, strategy='by_group', k=0.2)
models_results['uplift@20%'].append(sm_score)

In [114]:
models_results

{'approach': ['SoloModel'],
 'uplift@10%': [0.09479657086641838],
 'uplift@20%': [0.08615250896608034]}

In [117]:
from sklift.models import ClassTransformation

In [118]:
ct = ClassTransformation(CatBoostClassifier(iterations=100, thread_count=10, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, train_treat, estimator_fit_params={'cat_features':cat_features})

uplift_ct = ct.predict(X_test)

ct_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=test_treat, strategy='by_group', k=0.1)
models_results['approach'].append('ClassTransformation')
models_results['uplift@10%'].append(ct_score)
ct_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=test_treat, strategy='by_group', k=0.2)
models_results['uplift@20%'].append(ct_score)

In [119]:
models_results

{'approach': ['SoloModel', 'ClassTransformation'],
 'uplift@10%': [0.09479657086641838, 0.11074239336659406],
 'uplift@20%': [0.08615250896608034, 0.10488755635351597]}

In [120]:
from sklift.models import TwoModels

In [121]:
tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=100, thread_count=10, random_state=42, silent=True), 
    estimator_ctrl=CatBoostClassifier(iterations=100, thread_count=10, random_state=42, silent=True), 
    method='vanilla'
)
tm = tm.fit(
    X_train, y_train, train_treat,
    estimator_trmnt_fit_params={'cat_features': cat_features}, 
    estimator_ctrl_fit_params={'cat_features': cat_features}
)


In [122]:
uplift_tm = tm.predict(X_test)

tm_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=test_treat, strategy='by_group', k=0.1)
models_results['approach'].append('TwoModels')
models_results['uplift@10%'].append(tm_score)
tm_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=test_treat, strategy='by_group', k=0.2)
models_results['uplift@20%'].append(tm_score)

6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей

In [124]:
pd.DataFrame(data=models_results)

,approach,uplift@10%,uplift@20%
0,SoloModel,0.094797,0.086153
1,ClassTransformation,0.110742,0.104888
2,TwoModels,0.097637,0.080502
